In [23]:
import asyncio
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()
import os
from anthropic import AsyncAnthropic
from xml.sax.saxutils import escape
import re
import json

In [31]:
prompt = """\n\n
The above is a transcript of a lecture video with timestamps for each sentence in seconds.

Your task is to create a 30 second engaging and educational tiktok script for one topic in the video. 
Choose one of the more obscure and interesting topics from the script.
The tiktok should incorporate an engaging story or example.
Structure it similar to viral tiktoks you've seen before.

You will also need to give descriptions of pictures that will accompany the script.
You will also need to generate a description of the music that will accompany the script.
Output only a parsable JSON object with the following structure:
```
{
    "script": "This is the script of the tiktok",
    "music_description": "This is a short description of the music (eg. intense violin solo, peaceful orchestral piano)",
    "pictures": [
        {
            "description": "A picture of a cat",
            "timestamp": 0 (second)
        },
        {
            "description": "A picture of a dog",
            "timestamp": 5 (second))
        }
    ]
}
```
"""

In [32]:
# Load transcript from data/transcript.txt
with open("data/transcript.txt", "r") as f:
    transcript = f.read()

In [33]:
async def generate_script(text):
    client = AsyncAnthropic(
        # This is the default and can be omitted
        api_key=os.environ.get("ANTHROPIC_API_KEY"),
    )

    message = await client.messages.create(
        temperature=0,
        max_tokens=512,
        messages=[
            {
                "role": "user",
                "content": text + prompt,
            },
            {"role": "assistant", "content": "{"},
        ],
        model="claude-3-opus-20240229",
    )

    return message.content

response = asyncio.run(generate_script(transcript))

In [34]:
json_string = "{"+response[0].text
json_obj = json.loads(json_string)

with open("data/data.json", "w") as f:
    json.dump(json_obj, f, indent=4)